# Forward and backward

In this section, we will unveil the basic mechanism of the computational process of BigDL using a simple example. In this example, we show that how to obtain the gradients with a single forward and backward pass for updating.

We first need to import the necessary modules and initialize the engine.

In [1]:
import org.apache.log4j.{Level, Logger}
import org.apache.spark.SparkContext

import com.intel.analytics.bigdl.nn._
import com.intel.analytics.bigdl.utils.{Engine, LoggerFilter, T, Table}
import com.intel.analytics.bigdl.nn.{AbsCriterion}
import com.intel.analytics.bigdl.tensor._
import com.intel.analytics.bigdl.numeric.NumericFloat

Engine.init

Then we create a simple linear regression which can be formulized as *y = Wx + b*， where *W = [w1,w2]* are weight parameters and *b* is the bias.

In [2]:
// the input data size is 2*1, the output size is 1*1
val linear = Linear(2, 1)
// print the randomly initialized parameters
val (param1, param2) = linear.parameters()
println("Weight and Bias:")
param1.foreach(println)
println("GradWeight and gradBias:")
param2.foreach(println)

Weight and Bias:
0.3978135	-0.2532979	
[com.intel.analytics.bigdl.tensor.DenseTensor of size 1x2]
0.66053367
[com.intel.analytics.bigdl.tensor.DenseTensor of size 1]
GradWeight and gradBias:
0.0	0.0	
[com.intel.analytics.bigdl.tensor.DenseTensor of size 1x2]
0.0
[com.intel.analytics.bigdl.tensor.DenseTensor of size 1]


In [3]:
val input = Tensor(T(T(1f, -2f)))
// forward to output
val output = linear.updateOutput(input)
print(output.valueAt(1, 1))

1.564943

Then we backpropagate the error of the predicted output to the input.

In [4]:
// mean absolute error
val mae = AbsCriterion()
val target = Tensor(1).fill(0)

val loss = mae.updateOutput(output, target)
printf("loss: %s\n", loss.toString)
        
val gradOutput = mae.updateGradInput(output, target)
linear.updateGradInput(input, gradOutput)

val (param1, param2) = linear.parameters()
println("Weight and Bias:")
param1.foreach(println)
println("GradWeight and gradBias:")
param2.foreach(println)

loss: 1.564943
Weight and Bias:
0.3978135	-0.2532979	
[com.intel.analytics.bigdl.tensor.DenseTensor of size 1x2]
0.66053367
[com.intel.analytics.bigdl.tensor.DenseTensor of size 1]
GradWeight and gradBias:
0.0	0.0	
[com.intel.analytics.bigdl.tensor.DenseTensor of size 1x2]
0.0
[com.intel.analytics.bigdl.tensor.DenseTensor of size 1]


Finally, the Spark should be stopped.

In [5]:
sc.stop()

From above we can see that the backward pass has computed the gradient of the weights in respect to the loss. Therefore we can update the weights with the gradients using algorithms such as *stochastic gradient descent*. However in practice you **should** use *optimizer.optimize()* to circumvent the details.